입력 데이터 (Week 6/input.zip) 텍스트데이터: input.txt  

문제  

출현하는 단어의 첫 번째 두 글자 (= 길이2짜리 머리글자)의 출현 빈도를 세고     
이 출현빈도가 정확히 32인 "길이 2짜리 머리글자"로 시작하는 단어들을 중복없이 나열하였을 때 그 개수는?  
  
예를 들어 "active action accuracy behave behave behave "라는 문서가 있다면  
첫번째 두글자의 출현 빈도는 (ac, 3), (be, 3) 가 될테고,  
이 중 예를 들어 출현빈도가 정확히 3인 "두글자"로 시작하는 단어들의 리스트는 active action accuracy behave behave behave 가 될것이고  
이를 중복없이 나열하였을 때 그 개수는 4이다.  
힌트: groupByKey를 써서 두글자의 카운트를 세고   
      flatMap을 써서 출현빈도가 32인 길이 2인 머리글자를 가진 단어를 리스트한다음   
      count 가능  
주의  
대소문자 구별하지 않음  
문장부호(, . ? ‘ “ 등)은 따로 처리하지 않음      
* 처음 34 나옴 : 대소문자 구별을 datas에서 하지 않고 슬라이싱이후에 했음  
* 저번 quiz-3에서 못 풀어었던 첫 2글자가 들어가는 datas filtering을 datas.filter(lambda x: x[:2] in list)로 해결  

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext("local", "transformation 2")

In [41]:
# flatMap, filter이용 공백단어는 제거 
readfile=sc.textFile("input.txt").flatMap(lambda x: x.split(" ")).filter(lambda x: x !="")

# 단어중 첫글자에 공백 있을 수 있어 제외  # 1문자인 것은 제외, 두 글자이상만  # 대소문자구별하지 않으므로 소문자로 변경
datas=readfile.map(lambda x: x.strip()).filter(lambda x: len(x)>=2).map(lambda x: x.lower())

# 첫번째 두글자만 추출(slicing) 한 list 생성
keys = datas.map(lambda x : x[:2])

# groupByKey() 적용
keys_tuple_groupbykey = keys_lower.map(lambda x:(x, 1)).groupByKey().mapValues(sum)
# keys_tuple_groupbykey.collect()
#     [('st', 515),
#      ('in', 1076),
#      ('sc', 70),
#      ('pa', 188),
#      ('i.', 9),
#      ('(b', 1),
#      ('re', 456),
#      ('fr', 270),
#      ('th', 4546),,,,]

# reduceByKey() 적용 동일한 값
# keys_tuple_reducebykey = keys_lower.map(lambda x:(x, 1)).reduceByKey(lambda a,b : a+b)

# 출현빈도가 정확히 32인 "길이 2짜리 머리글자" filter 적용
keys_filtered_32 = keys_tuple_groupbykey.filter(lambda v: v[1] == 32)
print(keys_filtered_32.collect())
# [('em', 32), ('ce', 32)]
keys_32 = keys_filtered_32.keys()
keys_32_list=keys_32.collect()
type(keys_32_list) # list
keys_32_list
# ['em', 'ce']

[('em', 32), ('ce', 32)]


['em', 'ce']

In [42]:
# datas중에서 첫글자가 ['em', 'ce'] 시작하는 단어 filter
filtered = datas.filter(lambda x: x[:2] in keys_32_list)
# filtered.collect()
# ['emaciated',
#  'cesspool',
#  'certainly',
#  'certainly',
#  'employed',
#  'certainly',
#  'certain',
#  'century',
#  'empty',
#  'century.',
#  'certainly',
#  'certain',
#  'empty',
#  'empty',,,,]

# 중복을 제거한 값 count
filtered.distinct().count()

35